In [1]:
#q1
!hdfs dfsadmin -fs hdfs://boss:9000 -report

import urllib.request

url = "https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv"
filename = "hdma-wi-2021.csv"

urllib.request.urlretrieve(url, filename)

# Remove the files if they already exist in HDFS
!hdfs dfs -rm -f hdfs://boss:9000/single.csv
!hdfs dfs -rm -f hdfs://boss:9000/double.csv

# Upload the file to HDFS with a blockSize of 1MB
!HADOOP_user=hdfs hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -put hdma-wi-2021.csv hdfs://boss:9000/single.csv
!HADOOP_user=hdfs hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -put hdma-wi-2021.csv hdfs://boss:9000/double.csv

# Some of the source codes are from Microsoft Bing
# We got help from Piazza, @457, 458, 475

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 37545238528 (34.97 GB)
DFS Remaining: 37545189376 (34.97 GB)
DFS Used: 49152 (48 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.18.0.4:9866 (project-4-junho_4-dn-2.project-4-junho_4_default)
Hostname: 33e70ca9cb82
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 7031656448 (6.55 GB)
DFS Remaining: 18772594688 (17.48 GB)
DFS Used%: 0.00%
DFS Remaining%: 72.70%
Config

In [2]:
#q2
!hdfs dfs -du -h hdfs://boss:9000/double.csv hdfs://boss:9000/single.csv

166.8 M  333.7 M  hdfs://boss:9000/double.csv
166.8 M  166.8 M  hdfs://boss:9000/single.csv


In [3]:
#q3
import requests
import json

# WebHDFS URL for the file status operation
url = "http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS"

# Send a GET request to the WebHDFS
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Convert the response content to JSON
    file_status = response.json()
    
file_status


{'FileStatus': {'accessTime': 1698449356157,
  'blockSize': 1048576,
  'childrenNum': 0,
  'fileId': 16386,
  'group': 'supergroup',
  'length': 174944099,
  'modificationTime': 1698449362324,
  'owner': 'root',
  'pathSuffix': '',
  'permission': '644',
  'replication': 1,
  'storagePolicy': 0,
  'type': 'FILE'}}

In [4]:
#q4
import json

# WebHDFS URL for the file
url = "http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true"

# Send a GET request
response = requests.get(url)

# Convert the response to JSON0
data = response.json()

# Get the redirect URL from the data
redirect_url = data['Location']
redirect_url

'http://02bb164686d4:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0'

In [5]:
#q5
import requests
import json

# Initialize a dictionary to store the block distribution
block_distribution = {}

#from OH
# WebHDFS URL for the file
url = "http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS"

# Send a GET request
response = requests.get(url)

# Convert the response to JSON
data = response.json()
# data
# Get the redirect URL from the data
# BlockLocations are from Bing
redirect_url = data['BlockLocations']['BlockLocation']
block_distribution = {}

for ru in redirect_url:
    host_id = ru['hosts'][0]  # Assuming each item has a single host ID in the 'hosts' list (from Bing)
    if host_id in block_distribution:
        block_distribution[host_id] += 1
    else:
        block_distribution[host_id] = 1
block_distribution
# # Extract the container ID from the redirect URL
# container_id = redirect_url.split('/')[2].split(':')[0]
# container_id
# # Update the block distribution dictionary
# block_distribution[container_id] = block_distribution.get(container_id, 0) + 1
# block_distribution


{'02bb164686d4': 83, '33e70ca9cb82': 84}

In [6]:
#q6
import pyarrow as pa
import pyarrow.fs

# Connect to HDFS
hdfs = pa.fs.HadoopFileSystem("boss", 9000)

# Open the file
with hdfs.open_input_file("hdfs://boss:9000/single.csv") as file:
    # Read the first 10 bytes of the file
    bytes = file.read_at(10, 0)
bytes

# help from OH (' ' happened)

2023-10-27 23:29:41,543 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


b'activity_y'

In [7]:
#q7
import io

# Initialize a counter for the lines that contain "Single Family"
count = 0
with hdfs.open_input_file("hdfs://boss:9000/single.csv") as file:
    # For each line in the file
    # Wrap the file in a BufferedReader and a TextIOWrapper
    # This allows us to loop over lines and convert bytes to text
    # We asked Bing to use BufferedReader
    f = io.TextIOWrapper(io.BufferedReader(file))
    for line in f:
        # If the line contains "Single Family"
        if "Single Family" in line:
            # Increment the counter
            count += 1
        
count

444874